In [1]:
import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), 'utils'))
from configuration import *
import torch
import pprint
import PrintedSpikingNN as pSNN
from utils import *

args = parser.parse_args([])
args.projectname = 'test111'
args.DATASET = 0
args.DEVICE = 'cpu'
args = FormulateArgs(args)

In [2]:
print(f'Training network on device: {args.DEVICE}.')
MakeFolder(args)

train_loader, datainfo = GetDataLoader(args, 'train')
valid_loader, datainfo = GetDataLoader(args, 'valid')
test_loader, datainfo = GetDataLoader(args, 'test')
pprint.pprint(datainfo)

Training network on device: cpu.
{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_time': 300,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}


In [3]:
SetSeed(args.SEED)

setup = f"model_pSNN_data_{datainfo['dataname']}_seed_{args.SEED:02d}.model"
print(f'Training setup: {setup}.')

msglogger = GetMessageLogger(args, setup)
msglogger.info(f'Training network on device: {args.DEVICE}.')
msglogger.info(f'Training setup: {setup}.')
msglogger.info(datainfo)

Training setup: model_pSNN_data_acuteinflammation_seed_00.model.


In [4]:
if os.path.isfile(f'{args.savepath}/{setup}'):
        print(f'{setup} exists, skip this training.')
        msglogger.info('Training was already finished.')
else:
    topology = [datainfo['N_feature']] + args.hidden + [datainfo['N_class']]
    msglogger.info(f'Topology of the network: {topology}.')

    psnn = pSNN.PrintedSpikingNeuralNetwork(topology, args).to(args.DEVICE)

In [5]:

lossfunction = pSNN.LFLoss(args).to(args.DEVICE)
optimizer = torch.optim.Adam(psnn.GetParam(), lr=args.LR)

In [6]:
if args.PROGRESSIVE:
    psnn, best = train_pnn_progressive(psnn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)
else:
    psnn, best = train_pnn(psnn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)

if best:
    if not os.path.exists(f'{args.savepath}/'):
        os.makedirs(f'{args.savepath}/')
    torch.save(psnn, f'{args.savepath}/{setup}')
    msglogger.info('Training if finished.')
else:
    msglogger.warning('Time out, further training is necessary.')

/Users/haibinzhao/miniconda3/envs/ML/lib/python3.8/site-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


| Epoch:      0 | Train loss: 6.9315e-01 | Valid loss: 6.9314e-01 | Train acc: 0.4714 | Valid acc: 0.6087 | patience:   0 | lr: 0.1 | Epoch time: 13.6 | Power: 4.17e-06 |
| Epoch:      1 | Train loss: 6.9315e-01 | Valid loss: 6.9314e-01 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   0 | lr: 0.1 | Epoch time: 13.9 | Power: 5.36e-06 |
| Epoch:      2 | Train loss: 6.9315e-01 | Valid loss: 6.9314e-01 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   0 | lr: 0.1 | Epoch time: 13.7 | Power: 2.00e-05 |
| Epoch:      3 | Train loss: 6.9314e-01 | Valid loss: 6.9313e-01 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   0 | lr: 0.1 | Epoch time: 14.3 | Power: 2.03e-05 |
| Epoch:      4 | Train loss: 6.9314e-01 | Valid loss: 6.9313e-01 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   0 | lr: 0.1 | Epoch time: 14.0 | Power: 3.72e-05 |
| Epoch:      5 | Train loss: 6.9314e-01 | Valid loss: 6.9312e-01 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   0 | lr: 0.1 | Epoch time:

KeyboardInterrupt: 

In [ ]:
n = 0
c = 0
plt.figure(figsize=(30,5))
plt.plot(X_train[n,c,:]/10, label='input')
plt.plot(spikes[n,c,:].detach().numpy(), label='spike')
plt.plot(memories[n,c,:].detach().numpy(), label='memory')
plt.legend()

In [ ]:
tmac = psnn.TemporalWeightedSum(N_channel, N_class)
tmac

In [ ]:
mactmac = tmac(spikes)
mactmac.shape

In [ ]:
torch.manual_seed(seed)
topology = [N_channel, 3, N_class]
snn = psnn.SpikingNeuralNetwork(topology, beta, threshold)
optimizer = torch.optim.Adam(snn.parameters(), lr=0.01)


In [ ]:
snn.ResetOutput(False, 1)
input = snn.model[0](X_train)
spikes,memories = snn.model[1](input)
n = 0
c = 0
plt.figure(figsize=(30,5))
plt.plot(input[n,c,:].detach().numpy(), label='input')
plt.plot(spikes[n,c,:].detach().numpy(), label='spike')
plt.plot(memories[n,c,:].detach().numpy(), label='memory')
plt.legend()

In [ ]:
torch.manual_seed(seed)
topology = [N_channel, 3, N_class]
snn = psnn.SpikingNeuralNetwork(topology, beta=torch.zeros(1), threshold=torch.ones(1))
optimizer = torch.optim.Adam(snn.parameters(), lr=0.1)
loss_fn = psnn.SNNLoss()

In [ ]:
for epoch in range(1000):
    y_pred_train = snn(X_train)
    loss_train = loss_fn(y_pred_train, y_train)
    acc_train = (y_pred_train.sum(2).argmax(dim=1) == y_train).float().mean()
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    print(f'epoch: {epoch:-8d} | train loss: {loss_train:.5e} | train acc: {acc_train:.4f}')


In [ ]:
idx = torch.randperm(N_train)[:100]

X_visual = X_train[idx,:,:]
y_visual = y_train[idx]

plt.figure(figsize=(30,5))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.plot(X_visual[i,0,:].detach().numpy(), label=f'{y_visual[i]}')
    plt.axis('off')
    plt.legend()